In [10]:
# !apt-get install p7zip
# !p7zip -d -f -k ./mercari-price-suggestion-challenge/train.tsv.7z
# !unzip -o ./mercari-price-suggestion-challenge/sample_submission_stg2.csv.zip
# !unzip -o ./mercari-price-suggestion-challenge/test_stg2.tsv.zip
# !p7zip -d -f -k ./mercari-price-suggestion-challenge/test.tsv.7z
# !pip install pandas
# !pip install sklearn
# !pip install h2o
# !git clone https://github.com/4paradigm/autox.git
# !pip install ./autox
# !pip install psutil


In [11]:
import numpy as np 
import pandas as pd 
# from nltk.corpus import stopwords
# from sklearn.model_selection import train_test_split
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.preprocessing import LabelBinarizer
# from scipy.sparse import hstack, csr_matrix
# from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_log_error
import re
import gc
import matplotlib.pyplot as plt
#H2O
import h2o
from h2o.estimators.gbm import H2OGradientBoostingEstimator

# Pandas setting to display more dataset rows and columns
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import time
import os
import psutil

def count_time(func):
    def int_time():
        start_time = time.time()
        func()
        over_time = time.time()
        total_time = over_time - start_time
        print("程序运行了%s秒" % total_time)
    return int_time

def count_info(func):
    def float_info():
        pid = os.getpid()
        p = psutil.Process(pid)
        info_start = p.memory_full_info().uss/1024
        func()
        info_end=p.memory_full_info().uss/1024
        print("程序占用了内存"+str(info_end-info_start)+"KB")
    return float_info

In [12]:
train = pd.read_csv("sub_train.csv")
test1 = pd.read_csv('sub_val.csv')

In [13]:
test = test1.drop(columns=['price'])

In [14]:
# kf = KFold(n_splits=3, random_state=1001,shuffle=True)
# for i, (train_index, val_index) in enumerate(kf.split(train)):
#     trn= train.iloc[train_index].reset_index()
#     val= train.iloc[val_index].reset_index()
# val = val.drop(columns=['index'])
# test = val.drop(columns=['price'])
# test = test.rename(columns={'train_id':'test_id'})
# val = val.rename(columns={'train_id':'test_id'})
# test.to_csv('test.csv',index=False)
# val.to_csv('val.csv',index=False)
# train.to_csv('train.csv',index=False)
# !source /etc/profile
# !java

In [15]:
h2o.init()
all_train = h2o.H2OFrame(train)
# all_test = h2o.H2OFrame(test1)
#Get columns names for Building H2O Models
target = 'price'
predictors = [f for f in all_train.columns if f not in ['price','train_id','test_id']]

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,2 hours 30 mins
H2O_cluster_timezone:,Asia/Shanghai
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.1.1
H2O_cluster_version_age:,22 days
H2O_cluster_name:,H2O_from_python_unknownUser_ftgn45
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,19.94 Gb
H2O_cluster_total_cores:,15
H2O_cluster_allowed_cores:,15
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [7]:
from h2o.automl import H2OAutoML

In [8]:
aml = H2OAutoML(max_models=1, seed=1331)
aml.train(x=predictors, y=target, training_frame=all_train)

AutoML progress: |
15:34:09.170: _train param, Dropping bad and constant columns: [name]

███████████████████████████████████████████████████████████████| (done) 100%
Model Details
H2OXGBoostEstimator :  XGBoost
Model Key:  XGBoost_1_AutoML_1_20220501_153409


Model Summary: 


,,number_of_trees
0,,45.00000




ModelMetricsRegression: xgboost
** Reported on train data. **

MSE: 897.655590886451
RMSE: 29.960901035957697
MAE: 14.009382812257158
RMSLE: NaN
Mean Residual Deviance: 897.655590886451

ModelMetricsRegression: xgboost
** Reported on cross-validation data. **

MSE: 1078.3157105311182
RMSE: 32.83771780332973
MAE: 14.821529435005798
RMSLE: NaN
Mean Residual Deviance: 1078.3157105311182

Cross-Validation Metrics Summary: 


,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,mae,14.82153,0.66708,15.50185,14.70646,13.77638,15.28104,14.84193
1,mean_residual_deviance,1078.31570,496.45440,1936.61490,980.89703,656.88590,955.93870,861.24207
2,mse,1078.31570,496.45440,1936.61490,980.89703,656.88590,955.93870,861.24207
3,r2,0.20854,0.06483,0.16140,0.28077,0.23791,0.12203,0.24057
4,residual_deviance,1078.31570,496.45440,1936.61490,980.89703,656.88590,955.93870,861.24207
5,rmse,32.24425,6.94841,44.00699,31.31928,25.62979,30.91826,29.34693
6,rmsle,0.64226,0.00000,NaN,NaN,0.64226,NaN,NaN



Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance
0,,2022-05-01 15:34:39,30.059 sec,0.00000,44.97970,25.91228,2023.17384
1,,2022-05-01 15:34:40,30.883 sec,5.00000,32.86678,14.18261,1080.22517
2,,2022-05-01 15:34:40,31.429 sec,10.00000,31.50802,14.47681,992.75525
3,,2022-05-01 15:34:41,31.873 sec,15.00000,31.03842,14.47391,963.38354
4,,2022-05-01 15:34:41,32.303 sec,20.00000,30.70874,14.34461,943.02655
5,,2022-05-01 15:34:41,32.784 sec,25.00000,30.39629,14.23485,923.93472
6,,2022-05-01 15:34:42,33.229 sec,30.00000,30.29298,14.20785,917.66470
7,,2022-05-01 15:34:42,33.784 sec,35.00000,30.22101,13.99972,913.30964
8,,2022-05-01 15:34:43,34.224 sec,40.00000,30.04179,14.03004,902.50918
9,,2022-05-01 15:34:43,34.691 sec,45.00000,29.96090,14.00938,897.65559



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,brand_name.Louis Vuitton,3636423.00000,1.00000,0.26615
1,item_condition_id,1320539.62500,0.36314,0.09665
2,category_name.Electronics/Computers & Tablets/Laptops & Netbooks,670378.68750,0.18435,0.04907
3,category_name.Electronics/Cell Phones & Accessories/Cell Phones & ...,573936.06250,0.15783,0.04201
4,brand_name.nan,558457.25000,0.15357,0.04087
5,category_name.Women/Women's Handbags/Backpack Style,529091.25000,0.14550,0.03872
6,shipping,395904.28125,0.10887,0.02898
7,brand_name.Air Jordan,341629.84375,0.09395,0.02500
8,category_name.Men/Men's Accessories/Watches,288718.40625,0.07940,0.02113
9,category_name.Men/Shoes/Fashion Sneakers,274412.59375,0.07546,0.02008



See the whole table with table.as_data_frame()


In [9]:
test = h2o.H2OFrame(test)
prediction = aml.leader.predict(test)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%


/root/miniconda3/lib/python3.8/site-packages/h2o/job.py:83: UserWarning: Test/Validation dataset column 'item_description' has levels not trained on: ["" MUST BE 18YRS OR OLDER TO PURCHASE" 8" Frosted Glass Water Pipe With Perc, Ice Catcher And Removable Stem 1- 5pc pack of Silver Screens", ""An elegant perfume that blends neroli, rose and ylang ylang to still the mind and enhance serenity. This is the floral and fruity half of Breath of God, with Exhale providing the smoky depth. Inhale opens with a melon splash, but has a distinctive floral and lemon note with a depth provided by sandalwood. We've used a jojoba oil base rather than an alcohol base to allow it to settle and linger. Notes: melon, neroli, rose, ylang ylang"", ""B-Pave Heart Chain Link Bracelet" 16cm gold colored charm bracelet with Juicy heart charm, never been worn. Still has original tag, originally [rm]", ""Be Enchanted" No trades", ""Fairly light" bare minerals powder. Used a couple times. Don't care for it anymore.

In [13]:
prediction[['predict']]

predict
47.4826
22.9235
17.7365
12.0939
61.6452
24.9096
17.3044
20.37
14.9629
33.8882


In [25]:
pred = prediction.as_data_frame()

In [28]:
from sklearn.metrics import mean_squared_error
RMSLE = np.sqrt(mean_squared_error(test1['price'], pred))

In [29]:
RMSLE

32.91419227715444

In [1]:
import time
import os
import psutil
import numpy as np 
import pandas as pd 

def count_time(func):
    def int_time():
        start_time = time.time()
        func()
        over_time = time.time()
        total_time = over_time - start_time
        print("程序运行了%s秒" % total_time)
    return int_time

def count_info(func):
    def float_info():
        pid = os.getpid()
        p = psutil.Process(pid)
        info_start = p.memory_full_info().uss/1024
        func()
        info_end=p.memory_full_info().uss/1024
        print("程序占用了内存"+str(info_end-info_start)+"KB")
    return float_info

In [2]:
df_train = pd.read_csv('sub_train.csv')

In [3]:
df_train.head(1)

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description
0,39376,Size 8 Boots,2,Women/Shoes/Boots,Sonoma,24.0,0,Excellent condition! Worn maybe once or twice


In [4]:
documents = df_train['brand_name'].to_list()

In [5]:
# documents

In [6]:
# doc[0]
doc_ids = list(range(len(documents)))

In [7]:
from collections import OrderedDict
import h2o
from h2o.information_retrieval.tf_idf import tf_idf

In [8]:
import h2o
from h2o.estimators import H2OWord2vecEstimator, H2OGradientBoostingEstimator
@count_time
@count_info
def calcu_h2o():
    h2o.init()
    input_frame = h2o.H2OFrame(OrderedDict([('DocID', doc_ids), ('Document', documents)]),
                                    column_types=['numeric', 'string'])
    STOP_WORDS = ["ax","i","you","edu","s","t","m","subject","can",
                  "lines","re","what","there","all","we","one","the",
                  "a","an","of","or","in","for","by","on","but","is",
                  "in","a","not","with","as","was","if","they","are",
                  "this","and","it","have","from","at","my","be","by",
                  "not","that","to","from","com","org","like","likes",
                  "so"]
    # Make the 'tokenize' function:
    def tokenize(sentences, stop_word = STOP_WORDS):
        tokenized = sentences.tokenize("\\W+")
        tokenized_lower = tokenized.tolower()
        tokenized_filtered = tokenized_lower[(tokenized_lower.nchar() >= 2) | (tokenized_lower.isna()),:]
        tokenized_words = tokenized_filtered[tokenized_filtered.grep("[0-9]",invert=True,output_logical=True),:]
        tokenized_words = tokenized_words[(tokenized_words.isna()) | (~ tokenized_words.isin(STOP_WORDS)),:]
        return tokenized_words

    words = tokenize(input_frame["Document"])

    # Build word2vec model:
    w2v_model = H2OWord2vecEstimator(sent_sample_rate = 0.0, epochs = 10)
    w2v_model.train(training_frame=words)

    # Find synonyms for the words "teacher":
    w2v_model.find_synonyms("teacher", count = 5)

    vecs = w2v_model.transform(words, aggregate_method = "AVERAGE")

In [9]:
calcu_h2o()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "17" 2021-09-14; OpenJDK Runtime Environment (build 17+35-2724); OpenJDK 64-Bit Server VM (build 17+35-2724, mixed mode, sharing)
  Starting server from /root/miniconda3/lib/python3.8/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpcujnrgu6
  JVM stdout: /tmp/tmpcujnrgu6/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpcujnrgu6/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Asia/Shanghai
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.1.1
H2O_cluster_version_age:,22 days
H2O_cluster_name:,H2O_from_python_unknownUser_467gkj
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,20 Gb
H2O_cluster_total_cores:,15
H2O_cluster_allowed_cores:,15
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
word2vec Model Build progress: |█████████████████████████████████████████████████| (done) 100%
程序占用了内存4312.0KB
程序运行了7.640567064285278秒
